In [1]:
import numpy as np

In [105]:
class DecisionTree:
    def __init__(self, depth=4):
        self.depth = depth
        
    def branching(self, X, Y):
        branch_criterie = lambda L, R: len(Y) * self.__H(Y) - len(L) * self.__H(L) - len(R) * self.__H(R)
        
        feature_indexes = np.random.choice(np.arange(X.shape[1]), int(np.sqrt(X.shape[1])), replace=False)
        max_ig = float('-inf')
        max_feature = None
        max_feature_value = None
        
        for i in feature_indexes:
            unique_features_values = np.unique(X[:, i])
            for k in unique_features_values:
                left = Y[X[:, i] <= k]
                right = Y[X[:, i] > k]
                
                ig = branch_criterie(left, right)
                if ig > max_ig:
                    max_ig = ig
                    max_feature = i
                    max_feature_value = k
        
        return max_ig, max_feature, max_feature_value
    
    def fit(self, X, Y):
        pass
    def __H(self, Y):
        values = np.unique(Y, return_counts=True)
        entropy = 0
        for count in values[1]:
            p = count / len(Y)
            entropy -= p * np.log2(p)

        return entropy

In [106]:
tree = DecisionTree()

tree.branching(np.array([[1, 2, 3, 4, 5, 6, 8, 1, 2],
               [2, 3, 5, 5, 6, 7, 2, 3, 4],
                        [2, 1, 2, 3, 4, 5,6, 3, 1]]), np.array([2, 3, 5]))

(2.7548875021634682, 6, 2)